# Overview
- nb007までは取り除いていた*Name*、*Ticket*、*Cabin*を特徴量として取り扱えるよう、データ処理を行う。

In [238]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [255]:
#df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/train.csv')   # Google Colabの場合はこちら
df_train = pd.read_csv('../data/train.csv')   # ローカルの場合はこちら
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [256]:
print(df_train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [257]:
# 'Embarked'の欠損値処理
# =======================

print('Before: \n%s\n' % df_train['Embarked'].value_counts())

# 欠損値は2つだけなので、最頻値('S')で埋めることとする
df_train['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode().iloc[0])

print('After: \n%s\n' % df_train['Embarked'].value_counts())
print(df_train.isnull().sum())

Before: 
S    644
C    168
Q     77
Name: Embarked, dtype: int64

After: 
S    646
C    168
Q     77
Name: Embarked, dtype: int64

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [258]:
# 'Age'の欠損値処理
# ==================

print(df_train.corrwith(df_train['Age']), '\n')

# 'Age'は'Pclass'と相関が高いため、'Pclass'と'Sex'でグループ分けし、各グループの中央値で置き換える
print(df_train.groupby(['Pclass', 'Sex'])['Age'].median(), '\n')
df_train['Age'] = df_train.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
print(df_train.isnull().sum())

PassengerId    0.036847
Survived      -0.077221
Pclass        -0.369226
Age            1.000000
SibSp         -0.308247
Parch         -0.189119
Fare           0.096067
dtype: float64 

Pclass  Sex   
1       female    35.0
        male      40.0
2       female    28.0
        male      30.0
3       female    21.5
        male      25.0
Name: Age, dtype: float64 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [90]:
# 特徴量をX,ラベルをyとして分離しNumpy配列にする
X = df_train_raw.drop(['Survived'], axis=1).values
y = df_train_raw['Survived'].values
print(X, '\n')

# 訓練用、テスト用にデータ分割する   # 本当は最初にする必要あり？
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)   # 訓練:テスト = 80:20

print('Label counts in y: [0 1] =', np.bincount(y))
print('Label counts in y_train: [0 1] =', np.bincount(y_train))
print('Label counts in y_test: [0 1] =', np.bincount(y_test))

[[1 3 'Braund, Mr. Owen Harris' ... 7.25 nan 'S']
 [2 1 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)' ... 71.2833
  'C85' 'C']
 [3 3 'Heikkinen, Miss. Laina' ... 7.925 nan 'S']
 ...
 [889 3 'Johnston, Miss. Catherine Helen "Carrie"' ... 23.45 nan 'S']
 [890 1 'Behr, Mr. Karl Howell' ... 30.0 'C148' 'C']
 [891 3 'Dooley, Mr. Patrick' ... 7.75 nan 'Q']] 

Label counts in y: [0 1] = [549 342]
Label counts in y_train: [0 1] = [439 273]
Label counts in y_test: [0 1] = [110  69]


In [21]:
# Passengerid, Name, Ticket, Cabin列を除いた特徴量を取得
train_data = train_data_raw.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Sex, Embarked列をone-hot encordし、それぞれ1列を削除する
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'], drop_first=True)
train_data_columns = train_data.columns.values

array([[ 3., 22.,  1., ...,  1.,  0.,  1.],
       [ 1., 38.,  1., ...,  0.,  0.,  0.],
       [ 3., 26.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 3., nan,  1., ...,  0.,  0.,  1.],
       [ 1., 26.,  0., ...,  1.,  0.,  0.],
       [ 3., 32.,  0., ...,  1.,  1.,  0.]])

In [34]:
# Pipeline: pl_scv
# 欠損値平均補完 / SVC
# =====================

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

pl_svc = make_pipeline(SimpleImputer(missing_values=np.nan, strategy='mean'),
                       StandardScaler(), SVC(random_state=21, max_iter=10000))

from sklearn.model_selection import GridSearchCV
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__kernel': ['poly', 'rbf', 'sigmoid'], 'svc__gamma': param_range}]
gs = GridSearchCV(estimator=pl_svc, param_grid=param_grid, scoring='accuracy', cv=10, refit=True, n_jobs=-1)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.8370500782472613
{'svc__C': 100.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}


In [38]:
bestclf = gs.best_estimator_
print('Test accuracy: %.3f' % bestclf.score(X_test, y_test))

Test accuracy: 0.788
